In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import re
import string
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
df = pd.read_csv("C:/Users/Asus/Desktop/Python Work/books.csv")
df.head()


,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [5]:
# 📃 Step 3: Preprocess Descriptions
def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    text = re.sub("\d+", "", text)
    text = " ".join([word for word in text.split() if word not in stopwords.words('english')])
    return text

# Apply cleaning on descriptions
df['clean_title'] = df['title'].apply(clean_text)

In [6]:
vectorizer = TfidfVectorizer(stop_words='english')
title_matrix = vectorizer.fit_transform(df['clean_title'])


In [7]:
def recommend_books_by_title(input_text, top_n=5):
    input_cleaned = clean_text(input_text)
    input_vec = vectorizer.transform([input_cleaned])
    similarity = cosine_similarity(input_vec, title_matrix).flatten()
    indices = similarity.argsort()[-top_n:][::-1]
    results = df.iloc[indices][['title', 'authors', 'average_rating']]
    return results


In [9]:
query = "Harry Potter" #Motivation #Hate #Love Story
print(f"Top recommendations for: '{query}'\\n")
print(recommend_books_by_title(query))


Top recommendations for: 'Harry Potter'\n
                                                  title  \
2100  The Harry Potter Collection 1-4 (Harry Potter,...   
3752       Harry Potter Collection (Harry Potter, #1-6)   
421            Harry Potter Boxset (Harry Potter, #1-7)   
23    Harry Potter and the Goblet of Fire (Harry Pot...   
22    Harry Potter and the Chamber of Secrets (Harry...   

                          authors  average_rating  
2100  J.K. Rowling, Mary GrandPré            4.66  
3752                 J.K. Rowling            4.73  
421                  J.K. Rowling            4.74  
23    J.K. Rowling, Mary GrandPré            4.53  
22    J.K. Rowling, Mary GrandPré            4.37  
